<a href="https://colab.research.google.com/github/zw2788/LocalMinimaConstruction/blob/main/DwrtXGradientW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
from typing import Tuple

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from IPython.display import Image
from torch.autograd import grad

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, custom_W_0, custom_b, custom_V_0, custom_c):
        super(SimpleNN, self).__init__()

        # Ensure that the custom weights are tensors
        custom_W_0 = torch.tensor(custom_W_0, dtype=torch.float64)
        custom_b = torch.tensor(custom_b, dtype=torch.float64)
        custom_V_0 = torch.tensor(custom_V_0, dtype=torch.float64)
        custom_c = torch.tensor(custom_c, dtype=torch.float64)

        # Set the custom weights and biases
        self.W_0 = nn.Parameter(custom_W_0)
        self.b = nn.Parameter(custom_b)
        self.V_0 = nn.Parameter(custom_V_0)
        self.c = nn.Parameter(custom_c)

    def forward(self, x):
        x = F.sigmoid(torch.add(torch.matmul(x, self.W_0), self.b))
        x = F.sigmoid(torch.add(torch.matmul(x, self.V_0), self.c))
        return x

# Example usage
#custom_W_0 = [[0.1, 0.2], [0.3, 0.4]]  # Replace with your own initial values
#custom_b = [0.1, 0.2]  # Replace with your own initial values
#custom_V_0 = [[0.1], [0.2]]  # Replace with your own initial values
#custom_c = [0.1]  # Replace with your own initial values


def calculate_second_order_grad(model, X_raw_torch, Y_torch):
    # Forward pass
    output = model(X_raw_torch)
    # Compute loss
    loss = -torch.mean(Y_torch * torch.log(output) + (1 - Y_torch) * torch.log(1 - output))
    # Compute gradients of the loss w.r.t. weights
    loss.backward(create_graph=True)
    # Combine and compute the norm of all gradients
    all_grads = torch.cat([param.grad.flatten() for param in model.parameters()])
    grad_norm = torch.norm(all_grads)
    # Compute the derivative of the grad_norm with respect to X
    second_order_grad = torch.autograd.grad(grad_norm, X_raw_torch, retain_graph=True)[0]
    return second_order_grad

def perturb_weights(model, max_deviation=0.01):
    with torch.no_grad():
        for param in model.parameters():
            std_dev = param.abs().mean() * max_deviation
            noise = torch.randn(param.size()) * std_dev
            param[:] = param + noise

def restore_weights(model, saved_state):
    with torch.no_grad():
        for name, param in model.named_parameters():
            param[:] = saved_state[name]

def perturb_data(X, max_deviation=0.01):
    """Perturb the data tensor X."""
    with torch.no_grad():
        std_dev = X.abs().mean() * max_deviation
        noise = torch.randn(X.size()) * std_dev
        X.add_(noise)

In [77]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/zw2788/LocalMinimaConstruction/main/Ex1.csv")

data.head()

# data , drop NaN values
X_raw,  Y, W_0, b, V_0, c = data[['x_2dvec']].dropna().values, data['y'].dropna().values, data[['W_0']].dropna().values, data[['b']].dropna().values, data[['V_0']].dropna().values, data[['c']].dropna().values

#convert string to array

X_raw = np.array([eval(s[0]) for s in X_raw])

W_0 = np.array([eval(s[0]) for s in W_0])

b = np.array([eval(s[0]) for s in b])

V_0 = np.array([eval(s[0]) for s in V_0])

c = np.array([eval(s[0]) for s in c])

# Standardize the input
# Leave blank to match the example in paper

# formatting
Y = Y.reshape((-1, 1))
print(X_raw)
print(Y)
print(W_0)
#print(X_raw.shape[0])
X_raw = torch.tensor(X_raw, requires_grad=True)
Y = torch.tensor(Y)
print(W_0, b, V_0, c)

[[ 2.8  0.4]
 [ 3.1  4.3]
 [ 0.1 -3.4]
 [-4.2 -3.3]
 [-0.5  0.2]
 [-2.7 -0.4]
 [-3.  -4.3]
 [-0.1  3.4]
 [ 4.2  3.2]
 [ 0.4 -0.1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]]
[[ 1.05954587 -0.05625762]
 [-0.03749863  1.09518945]]
[[ 1.05954587 -0.05625762]
 [-0.03749863  1.09518945]] [[-0.03749863  1.09518945]] [[ 3.76921058]
 [-3.72139955]] [[-0.0148436]]


In [ ]:
# Convert data to PyTorch tensors
X_raw_torch = torch.tensor(X_raw, requires_grad=True)
Y_torch = torch.tensor(Y)


# Set a threshold for the norm of the second-order gradient
threshold = 0.05 # Adjust this threshold as needed
max_iterations = 20  # Maximum number of iterations to prevent infinite loops

# Learning rate
learning_rate = 0.01

# Monte Carlo method sampling points
MC_num_samples = 200

# Surrouning points' grads' propotion
surrounding_propotion = 50

nn_model = SimpleNN(W_0, b, V_0, c)

#original_weights = W_0, b, V_0, c
original_weights = {
    'W_0': nn_model.W_0.data.clone(),
    'b': nn_model.b.data.clone(),
    'V_0': nn_model.V_0.data.clone(),
    'c': nn_model.c.data.clone()
}
print(original_weights)
print("Initial X_raw {}".format(X_raw_torch))
max_deviation_for_X = 0.02  # You can adjust this value as needed
perturb_data(X_raw_torch, max_deviation=max_deviation_for_X)
print("Perturbed X_raw {}".format(X_raw_torch))

for i in range(max_iterations):

    # Calculate the gradient at the central point
    central_grad = calculate_second_order_grad(nn_model, X_raw_torch, Y_torch)

    # Surrouning points' grads
    surrounding_grads = []


    # Calculate the gradient at the surrounding points by MC
    for _ in range(MC_num_samples):

      nn_model_sample = SimpleNN(custom_W_0=original_weights['W_0'],custom_b=original_weights['b'],custom_V_0=original_weights['V_0'],custom_c=original_weights['c'])
      #print("W_0 (before perturbation):", nn_model_sample.W_0.data)
      # Perturb weights
      perturb_weights(nn_model_sample, max_deviation=0.01)
      #print("W_0 (after perturbation):", nn_model_sample.W_0.data)
      # Calculate second-order gradient
      grad = calculate_second_order_grad(nn_model_sample, X_raw_torch, Y_torch)
      surrounding_grads.append(grad)

    #print("Surrounding grad {}".format(surrounding_grads))
    # Average surrounding gradients
    average_surrounding_grad = sum(surrounding_grads) / MC_num_samples

    # Combine gradients
    combined_grad = central_grad + surrounding_propotion * average_surrounding_grad

###############
    # Combine and compute the norm of all gradients
    #all_grads = torch.cat([nn_model.W_0.grad.flatten(), nn_model.V_0.grad.flatten(), nn_model.b.grad.flatten(), nn_model.c.grad.flatten()])
    #grad_norm = torch.norm(all_grads)

    # Compute the derivative of the grad_norm with respect to X
    #second_order_grad = torch.autograd.grad(grad_norm, X_raw_torch, retain_graph=True)[0]

    # Check if the norm of the second-order gradient is below the threshold
    if torch.norm(combined_grad) < threshold:
        print(f"Convergence reached at iteration {i}")
        break

    # Update X_raw using gradient descent
    X_raw_torch.data -= learning_rate * combined_grad

    # Zero out gradients for the next iteration
    nn_model.zero_grad()
    X_raw_torch.grad = None

# Print final modified data
#print(surrounding_grads)
#print("Final modified X_raw:")
print(X_raw_torch)

<ipython-input-68-9309a5caa323>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_raw_torch = torch.tensor(X_raw, requires_grad=True)
<ipython-input-68-9309a5caa323>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_torch = torch.tensor(Y)
<ipython-input-44-7093221fd565>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  custom_W_0 = torch.tensor(custom_W_0, dtype=torch.float64)
<ipython-input-44-7093221fd565>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().de

{'W_0': tensor([[ 1.0595, -0.0563],
        [-0.0375,  1.0952]], dtype=torch.float64), 'b': tensor([[-0.0375,  1.0952]], dtype=torch.float64), 'V_0': tensor([[ 3.7692],
        [-3.7214]], dtype=torch.float64), 'c': tensor([[-0.0148]], dtype=torch.float64)}
Initial X_raw tensor([[ 2.8000,  0.4000],
        [ 3.1000,  4.3000],
        [ 0.1000, -3.4000],
        [-4.2000, -3.3000],
        [-0.5000,  0.2000],
        [-2.7000, -0.4000],
        [-3.0000, -4.3000],
        [-0.1000,  3.4000],
        [ 4.2000,  3.2000],
        [ 0.4000, -0.1000]], dtype=torch.float64, requires_grad=True)
Perturbed X_raw tensor([[ 2.7485,  0.3702],
        [ 3.1686,  4.3364],
        [ 0.1450, -3.4219],
        [-4.1875, -3.2948],
        [-0.5174,  0.1783],
        [-2.7019, -0.4439],
        [-2.9534, -4.3165],
        [-0.1050,  3.3718],
        [ 4.1801,  3.1714],
        [ 0.4512, -0.1165]], dtype=torch.float64, requires_grad=True)
tensor([[ 2.8651,  0.1410],
        [ 3.2493,  4.3206],
        [ 0.

In [ ]:
# Re-executing the code to define the function for computing the Hessian matrix and its eigenvalues

def compute_hessian_and_eigenvalues(model, data, target):
    """
    Compute the Hessian matrix and its eigenvalues for the weights of a neural network model.

    :param model: The neural network model.
    :param data: Input data (X).
    :param target: Target data (Y).
    :return: Hessian matrix and its eigenvalues.
    """
    # Forward pass
    output = model(data)
    # Compute loss
    loss = -torch.mean(target * torch.log(output) + (1 - target) * torch.log(1 - output))

    # First-order gradients (w.r.t weights)
    first_order_grads = torch.autograd.grad(loss, model.parameters(), create_graph=True)

    # Flatten the first-order gradients
    grads_flatten = torch.cat([g.contiguous().view(-1) for g in first_order_grads])

    # Hessian computation
    hessian = []
    for grad in grads_flatten:
        # Compute second-order gradients (w.r.t each element in the first-order gradients)
        second_order_grads = torch.autograd.grad(grad, model.parameters(), retain_graph=True)

        # Flatten and collect the second-order gradients
        hessian_row = torch.cat([g.contiguous().view(-1) for g in second_order_grads])
        hessian.append(hessian_row)

    # Stack to form the Hessian matrix
    hessian_matrix = torch.stack(hessian)

    # Compute eigenvalues
    eigenvalues, _ = torch.linalg.eig(hessian_matrix)

    return hessian_matrix, eigenvalues

# Note: To use this function, you'll need to provide your neural network model, the input data (X), and the target data (Y).


In [ ]:
nn_model = SimpleNN(W_0, b, V_0, c)
compute_hessian_and_eigenvalues(nn_model, X_raw_torch, Y_torch)

(tensor([[ 0.1229, -0.0537,  0.1374, -0.0068,  0.0104, -0.0119,  0.0367,  0.0274,
           0.0183],
         [-0.0537,  0.2881, -0.0068,  0.1489, -0.0119, -0.0250, -0.0553, -0.0279,
          -0.0097],
         [ 0.1374, -0.0068,  0.4870, -0.0480, -0.0080,  0.0173,  0.0074,  0.0458,
          -0.0111],
         [-0.0068,  0.1489, -0.0480,  0.1303,  0.0173, -0.0501,  0.0052,  0.0031,
           0.0472],
         [ 0.0104, -0.0119, -0.0080,  0.0173,  0.0408, -0.0349,  0.0322,  0.0441,
           0.0712],
         [-0.0119, -0.0250,  0.0173, -0.0501, -0.0349,  0.0310, -0.0373, -0.0468,
          -0.0681],
         [ 0.0367, -0.0553,  0.0074,  0.0052,  0.0322, -0.0373,  0.0877,  0.0855,
           0.1066],
         [ 0.0274, -0.0279,  0.0458,  0.0031,  0.0441, -0.0468,  0.0855,  0.0956,
           0.1139],
         [ 0.0183, -0.0097, -0.0111,  0.0472,  0.0712, -0.0681,  0.1066,  0.1139,
           0.1931]], dtype=torch.float64),
 tensor([ 0.5579+0.j,  0.4084+0.j,  0.3788+0.j,  0.0837+0.j

# Set a threshold for the norm of the second-order gradient
threshold = 0.05 # Adjust this threshold as needed
max_iterations = 10  # Maximum number of iterations to prevent infinite loops

# Learning rate
learning_rate = 0.2

# Monte Carlo method sampling points
MC_num_samples = 10

# Surrouning points' grad
surrounding_grads = []

# parameters for the first layer
W_0 = np.array([[1.05954587,-0.05625762],[-0.03749863,1.09518945]])
b = np.array([[-0.050686,-0.06894291]])

# parameters for the second layer

V_0 = np.array([[3.76921058],[-3.72139955]])
c = np.array([[-0.0148436]])

nn_model = SimpleNN(W_0, b, V_0, c)

perturb_weights(nn_model, max_deviation=0.01)
restore_weights(nn_model, original_weights)  # Assuming perturb_weights is defined as before
print(perturb_weights)
K=calculate_second_order_grad(nn_model, X_raw_torch, Y_torch)
print(K)
print("W_0 (after perturbation):", nn_model.W_0.data)
print("b (after perturbation):", nn_model.b.data)
print("V_0 (after perturbation):", nn_model.V_0.data)
print("c (after perturbation):", nn_model.c.data)

# Forward pass
output = nn_model(X_raw_torch)

# Compute loss
loss = -torch.mean(Y_torch * torch.log(output) + (1 - Y_torch) * torch.log(1 - output))
print(loss)
# Compute gradients of the loss w.r.t. weights
loss.backward(create_graph=True)


# Combine and compute the norm of all gradients
all_grads = torch.cat([nn_model.W_0.grad.flatten(), nn_model.V_0.grad.flatten(), nn_model.b.grad.flatten(), nn_model.c.grad.flatten()])
print(all_grads)
grad_norm = torch.norm(all_grads)
print(grad_norm)
# Compute the derivative of the grad_norm with respect to X
second_order_grad = torch.autograd.grad(grad_norm, X_raw_torch, retain_graph=True)[0]
print(torch.norm(second_order_grad))
# If you want to perform gradient descent on X_raw
learning_rate = 0.01
#X_raw_torch.data -= learning_rate * second_order_grad